# Project 2

### Investigating the Importance of R&D Researchers on Innovation

Increased global demand for clean energy technology and other advanced manfuacturing has led to renewed global attention on how policy makers can support innovation and the development of intellectual property. Despite the current adminstration's focus on developing US industrial dominance, a number of key science agencies and grant recipients have seen funding cuts. 

To investigate the potential effects of this for the US, we will look at the historical relationship between number of R&D researchers and US patent applications over time.

Hypothesis: increases in the number of researchers in R&D is associated with higher numbers of patent applications. 

The data for this analysis comes from the two sources below: 

Source 1: CEIC data is used for the number of researchers engaged in Research & Development (R&D), expressed as researchers per million of population. The link to the data source is here: https://insights-ceicdata-com.ezproxy.cul.columbia.edu/Name-your-insight/views

Source 2: World Bank data is used for the number of patent applications by US residents. The link to the data source is here: https://data360.worldbank.org/en/indicator/WB_WDI_IP_PAT_RESD

### Step 1: Load and clean the datasets 

In [54]:
import pandas as pd
import plotly.express as px

In [55]:
import plotly.express as px
from IPython.display import HTML

In [56]:
#Read CEIC csv and show first 30 headings below given initial rows of metadata
researchers_df = pd.read_csv("researchersUS.csv")
researchers_df.head(30)

,Unnamed: 0,Researchers in R&D: per Million People
0,Region,United States
1,Subnational,NaN
2,Frequency,"Annual, ending ""Dec"" of each year"
3,Unit,Ratio
4,Source,World Bank
5,Status,Active
6,Series ID,274038702
7,SR Code,SR5357800
8,Trade Code,NaN
9,Mnemonic,NaN


In [57]:
#Remove initial metadata rows and rename headings
researchers_df = pd.read_csv("researchersUS.csv", skiprows=33, header=None, names=['Year', 'Researchers per Million Population'])

#Start data set from 2000 to align with World Bank dataset
researchers_df = researchers_df[researchers_df['Year'] >= 2000]
researchers_df

,Year,Researchers per Million Population
0,2000,3487.83262
1,2001,3529.26596
2,2002,3576.98706
3,2003,3779.03771
4,2004,3637.14002
5,2005,3560.28174
6,2006,3596.55336
7,2007,3543.24508
8,2008,3668.58989
9,2009,3790.43930


In [58]:
#Read World Bank CSV and show initial rows
world_patents_df = pd.read_csv("worldpatents.csv")
world_patents_df.head()

,FREQ,FREQ_LABEL,REF_AREA,REF_AREA_LABEL,INDICATOR,INDICATOR_LABEL,UNIT_MEASURE,UNIT_MEASURE_LABEL,AGG_METHOD,AGG_METHOD_LABEL,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,A,Annual,AFG,Afghanistan,WIPO_ICT_PAT_PUB_TOT,WIPO - Number of ICT patent publications: all ...,PATN,Patents,MEAN,Mean,...,1.0,NaN,NaN,NaN,NaN,1.0,4.0,NaN,NaN,NaN
1,A,Annual,AGO,Angola,WIPO_ICT_PAT_PUB_TOT,WIPO - Number of ICT patent publications: all ...,PATN,Patents,MEAN,Mean,...,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN
2,A,Annual,ALB,Albania,WIPO_ICT_PAT_PUB_TOT,WIPO - Number of ICT patent publications: all ...,PATN,Patents,MEAN,Mean,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0
3,A,Annual,AND,Andorra,WIPO_ICT_PAT_PUB_TOT,WIPO - Number of ICT patent publications: all ...,PATN,Patents,MEAN,Mean,...,2.0,1.0,3.0,NaN,1.0,6.0,2.0,5.0,3.0,NaN
4,A,Annual,ANT,Netherlands Antilles,WIPO_ICT_PAT_PUB_TOT,WIPO - Number of ICT patent publications: all ...,PATN,Patents,MEAN,Mean,...,2.0,5.0,3.0,7.0,1.0,2.0,NaN,3.0,3.0,NaN


Our data includes patent applications for all countries as well as unecessary columns and metadata. We need to clean this dataframe to only show United States and to only show patent applications

In [59]:
#Filter for only United States
us_patents_df = world_patents_df[world_patents_df['REF_AREA_LABEL'] == 'United States']
us_patents_df

,FREQ,FREQ_LABEL,REF_AREA,REF_AREA_LABEL,INDICATOR,INDICATOR_LABEL,UNIT_MEASURE,UNIT_MEASURE_LABEL,AGG_METHOD,AGG_METHOD_LABEL,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
170,A,Annual,USA,United States,WIPO_ICT_PAT_PUB_TOT,WIPO - Number of ICT patent publications: all ...,PATN,Patents,MEAN,Mean,...,119767.0,140046.0,132536.0,142111.0,139121.0,132923.0,137523.0,140499.0,142735.0,148239.0


In [60]:
#Remove uneccessary columns and transform from wide to long format. Drop the final year to align with timeline for researchers_df 
patents_by_year = (
    us_patents_df.drop(columns=["FREQ", "FREQ_LABEL", "REF_AREA", "REF_AREA_LABEL", "INDICATOR", 
                              "INDICATOR_LABEL", "UNIT_MEASURE", "UNIT_MEASURE_LABEL", "AGG_METHOD", 
                              "AGG_METHOD_LABEL", "DATABASE_ID", "DATABASE_ID_LABEL", "UNIT_MULT", 
                              "UNIT_MULT_LABEL", "OBS_STATUS", "OBS_STATUS_LABEL", "OBS_CONF", "OBS_CONF_LABEL"])
    .melt(
        var_name="Year",
        value_name="Patent Applications"
    )
    .dropna()
    .iloc[:-1]  # Remove the last row
)

#View new dataframe  
patents_by_year

,Year,Patent Applications
0,2000,46665.0
1,2001,60922.0
2,2002,81398.0
3,2003,84454.0
4,2004,89419.0
5,2005,93767.0
6,2006,95930.0
7,2007,99946.0
8,2008,108728.0
9,2009,104584.0


### Step 2: Visualize cleaned dataframes 

In [ ]:
#Visualize with a line graph R&D researchers through time
fig_researchers = px.line(researchers_df,
                                 x="Year",
                                 y="Researchers per Million Population",
                                 title="US Researchers in R&D: per Million Population")

HTML(fig_researchers.to_html(include_plotlyjs="cdn", full_html=False))

R&D researchers per million of population has been increasing over time, with a significant spike post 2015. One potential explanation for this may be the significant focus on clean technology post the 2015 Paris agreement as well as the increasing focus on semiconductor manufacturing and artificial intelligence from around that time. The spike post COVID in 2020 could also be related to vaccine science.

In [62]:
# We can better visualize the change from 2015 and 2020 by adding vertical lines at those years

# Add vertical line at 2015
fig_researchers.add_vline(x=2015, line_width=2, line_dash="dash", line_color="red")

# Add vertical line at 2020
fig_researchers.add_vline(x=2020, line_width=2, line_dash="dash", line_color="green")

HTML(fig_researchers.to_html(include_plotlyjs="cdn", full_html=False))

Now let's move on to patent applications

In [63]:
#Visualize with a line graph patent applications through time
fig_patents = px.line(patents_by_year,
                                 x="Year",
                                 y="Patent Applications",
                                 title="US Patent Applications")
HTML(fig_patents.to_html(include_plotlyjs="cdn", full_html=False))

This graph shows a fairly steady increase in patent applications up until 2015 with some plateau from 2015 onwards

### Step 3: Combine researcher and patent visuals

In [50]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [64]:
#Create a line graph that shows both researchers and patent applications on the same graph

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

#Add researchers and patent data
fig.add_trace(
    go.Scatter(
        x=researchers_df["Year"],
        y=researchers_df["Researchers per Million Population"],
        name="Researchers per Million Population",
    ),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(
        x=patents_by_year["Year"],
        y=patents_by_year["Patent Applications"],
        name="Patent Applications",
    ),
    secondary_y=True,
)

# Add title
fig.update_layout(title_text="Researchers vs. Patent Applications, United States")

# Set x-axis title
fig.update_xaxes(title_text="Year")

# Set y-axes titles
fig.update_yaxes(title_text="Researchers per Million of Population", secondary_y=False)
fig.update_yaxes(title_text="Patent Applications", secondary_y=True)

HTML(fig.to_html(include_plotlyjs="cdn", full_html=False))

The above graph shows a positive relationship between R&D researchers and patent applications. However, it is not a strong relationship and following the post 2015 uptick in researchers, we did not see a commensurate increase in patent applications. In order to better investigate the relationship between these two variables, we may want to narrow our data set to specific sectors or try and control for changing industry dynamics, policy settings etc. 

Overall, from this analysis alone we can draw the limited conclusion that reducing government support for R&D is likely to have some negative impact on US innovation but further analysis would be required to determine the nature and magnitude of this negative effect.